In [79]:
import pandas as pd
import csv

## Data Preprocessing

In [80]:
rows = []
csv_path = 'data/music4all_subset/id_genres.csv'
with open(csv_path, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    header = next(csv_reader)[0].split('\t')
    print(header)
    for row in csv_reader:
        temp = row[0].split('\t')
        if len(row) > 2:
            temp.extend(row[2:])
        rows.append(temp)

['id', 'genres']


In [81]:
header = ['id', 'genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7']

In [82]:
df = pd.DataFrame(rows, columns=header)
df

,id,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,0009fFIM1eYThaPg,pop,None,None,None,None,None,None
1,0010xmHR6UICBOYT,underground hip hop,None,None,None,None,None,None
2,002Jyd0vN4HyCpqL,hard rock,classic rock,None,None,None,None,None
3,006TYKNjNxWjfKjy,symphonic metal,symphonic power metal,None,None,None,None,None
4,007LIJOPQ4Sb98qV,post-punk,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
109264,zzyyPUs7hC9Nz2e1,hardcore punk,hardcore,None,None,None,None,None
109265,zzz0n04uuTUA7fNh,pop,None,None,None,None,None,None
109266,zzzj3LYaZtYtbzSr,singer-songwriter,None,None,None,None,None,None
109267,zzznMjZAKnJJXQSj,synthpop,pop,None,None,None,None,None


In [85]:
melt = pd.melt(df, id_vars=['id'], value_vars=df.columns[1:])
allocc = melt['value'].value_counts()

top11occ = melt['value'].value_counts().head(16)
top11occ

pop                  16810
rock                 15947
electronic            8855
indie rock            5730
alternative rock      5292
folk                  4559
soul                  3985
classic rock          3809
metal                 3639
rap                   3623
singer-songwriter     3445
experimental          2980
hard rock             2947
indie pop             2810
post-hardcore         2506
punk                  2458
Name: value, dtype: int64

In [87]:
top11genres = set(top11occ.keys())
top11genres.remove('rock')
top11genres.remove('post-hardcore')
top11genres.remove('hard rock')
top11genres.remove('indie pop')
top11genres.remove('experimental')

print(top11genres)
print(len(top11genres))

{'indie rock', 'soul', 'punk', 'folk', 'metal', 'electronic', 'pop', 'singer-songwriter', 'alternative rock', 'rap', 'classic rock'}
11


In [88]:
users_with_top11 = melt[melt['value'].isin(top11genres)]['id'].drop_duplicates()

In [89]:
filtered_users = df[df['id'].isin(users_with_top11)]

In [90]:
filtered_users

,id,genre1,genre2,genre3,genre4,genre5,genre6,genre7
0,0009fFIM1eYThaPg,pop,None,None,None,None,None,None
2,002Jyd0vN4HyCpqL,hard rock,classic rock,None,None,None,None,None
5,00CH4HJdxQQQbJfu,indie rock,shoegaze,experimental,None,None,None,None
6,00DZ3XCAQb2FdCc6,ambient,electronic,None,None,None,None,None
8,00IeldeA9ijZOL0P,pop,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
109263,zzyb5LvKJTWLVnrk,soul,mpb,funk,None,None,None,None
109265,zzz0n04uuTUA7fNh,pop,None,None,None,None,None,None
109266,zzzj3LYaZtYtbzSr,singer-songwriter,None,None,None,None,None,None
109267,zzznMjZAKnJJXQSj,synthpop,pop,None,None,None,None,None


In [92]:
final_df = pd.DataFrame()
final_df['id'] = filtered_users['id']

In [93]:
def prioritize_pop(row):
    if 'pop' in row and len(set(row) & set(top11genres)) > 1:
        return next((x for x in row if x in top11genres and x != 'pop'), None)
    else:
        return next((x for x in row if x in top11genres), None)
    
#final_df['genre'] = df.iloc[:, 1:].apply(prioritize_pop, axis=1)
final_df['genre'] = df.iloc[:, 1:].apply(lambda row: next((x for x in row if x in top11genres), None), axis=1)

In [94]:
final_df

,id,genre
0,0009fFIM1eYThaPg,pop
2,002Jyd0vN4HyCpqL,classic rock
5,00CH4HJdxQQQbJfu,indie rock
6,00DZ3XCAQb2FdCc6,electronic
8,00IeldeA9ijZOL0P,pop
...,...,...
109263,zzyb5LvKJTWLVnrk,soul
109265,zzz0n04uuTUA7fNh,pop
109266,zzzj3LYaZtYtbzSr,singer-songwriter
109267,zzznMjZAKnJJXQSj,pop


In [95]:
final_counts = final_df['genre'].value_counts()
final_counts

pop                  15239
electronic            7703
indie rock            5006
alternative rock      4465
folk                  3982
soul                  3544
rap                   3395
metal                 3357
classic rock          3268
singer-songwriter     2415
punk                  2203
Name: genre, dtype: int64

In [ ]:
rows = []
csv_path = 'data/music4all_subset/id_lang.csv'
with open(csv_path, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    
    header = next(csv_reader)[0].split('\t')
    print(header)
    for row in csv_reader:
        temp = row[0].split('\t')
        if len(row) > 2:
            temp.extend(row[2:])
        rows.append(temp)

In [ ]:
df_filter_en = pd.DataFrame(rows, columns=['id', 'lang'])
fin_fil_df = final_df[final_df['id'].isin(df_filter_en[df_filter_en['lang'] == 'en']['id'])]

In [ ]:
fin_fil_df